In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

# =========================
# 1. Cargar archivo crudo
# =========================

import pandas as pd

raw = pd.read_csv(
    'terraclimate_features_training.csv',
    header=None,
    names=['Latitude','Longitude','Date','pet']
)

# Conversión robusta de tipos
raw['Latitude']  = pd.to_numeric(raw['Latitude'],  errors='coerce')
raw['Longitude'] = pd.to_numeric(raw['Longitude'], errors='coerce')
raw['pet']       = pd.to_numeric(raw['pet'],       errors='coerce')

# Fechas: formato explícito para evitar el warning y garantizar consistencia
raw['Date'] = pd.to_datetime(raw['Date'], format='%d-%m-%Y', errors='coerce')

# Elimina solo filas inválidas
df = raw.dropna(subset=['Latitude','Longitude','pet','Date']).copy()
# Intento de conversión
lat = pd.to_numeric(raw['Latitude'], errors='coerce')
lon = pd.to_numeric(raw['Longitude'], errors='coerce')
pet = pd.to_numeric(raw['pet'], errors='coerce')
date = pd.to_datetime(raw['Date'], dayfirst=True, errors='coerce')

# Filas problemáticas
mask = lat.isna() | lon.isna() | pet.isna() | date.isna()
problematic_rows = raw[mask]

print("\n=== FILAS PROBLEMÁTICAS DETECTADAS ===")
print(f"Total filas dañadas: {mask.sum()}")
print(problematic_rows.head())

# =========================
# 2. Limpieza del dataset
# =========================
df = raw.copy()
df['Latitude'] = lat
df['Longitude'] = lon
df['pet'] = pet
df['Date'] = date

df = df.dropna()

print("\n=== REGISTROS FINALES DESPUÉS DE LIMPIEZA ===")
print(f"Total final de filas limpias: {len(df)}")

# =========================
# 3. Estadísticas descriptivas
# =========================
stats = df.describe()
print("\n=== ESTADÍSTICAS DESCRIPTIVAS ===")
print(stats)

# =========================
# 4. Correlación
# =========================
corr = df[['Latitude','Longitude','pet']].corr()
print("\n=== MATRIZ DE CORRELACIÓN ===")
print(corr)

# =========================
# 5. Gráficas
# =========================

# Time series
ts = df.groupby('Date')['pet'].mean()
plt.figure(figsize=(10,5))
plt.plot(ts)
plt.title('PET promedio en el tiempo')
plt.xlabel('Fecha')
plt.ylabel('PET')
plt.grid(True)
plt.tight_layout()
plt.savefig('pet_tiempo.png')
plt.close()

# Histograma
plt.figure(figsize=(8,5))
sns.histplot(df['pet'], bins=40, kde=True)
plt.title('Distribución de PET')
plt.xlabel('PET')
plt.ylabel('Frecuencia')
plt.savefig('pet_hist.png')
plt.close()

# Geospatial scatter
plt.figure(figsize=(8,6))
plt.scatter(df['Longitude'], df['Latitude'], c=df['pet'], cmap='viridis', s=10)
plt.colorbar(label='PET')
plt.title('Distribución espacial de PET')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.savefig('pet_geo.png')
plt.close()



=== FILAS PROBLEMÁTICAS DETECTADAS ===
Total filas dañadas: 1
   Latitude  Longitude Date  pet
0       NaN        NaN  NaT  NaN

=== REGISTROS FINALES DESPUÉS DE LIMPIEZA ===
Total final de filas limpias: 9319

=== ESTADÍSTICAS DESCRIPTIVAS ===
          Latitude    Longitude                           Date          pet
count  9319.000000  9319.000000                           9319  9319.000000
mean    -28.474988    26.868414  2013-08-06 06:31:33.679579392   175.166082
min     -34.405833    17.730278            2011-01-02 00:00:00    52.700000
25%     -30.160091    26.126667            2012-06-07 00:00:00   156.100000
50%     -28.058889    27.409060            2013-08-28 00:00:00   172.500000
75%     -26.861111    29.245556            2014-10-15 00:00:00   193.100000
max     -22.225556    32.325000            2015-12-31 00:00:00   270.800020
std       2.760282     3.535164                            NaN    29.469867

=== MATRIZ DE CORRELACIÓN ===
           Latitude  Longitude       pe

In [4]:
# ============================================================
# MAPA DE TERRACLIMATE BASADO EN PET (Evapotranspiración Potencial)
# ============================================================

import pandas as pd
import folium

# --- 1. Cargar dataset --------------------------------------

df = pd.read_csv("terraclimate_features_training.csv")
df["Sample Date"] = pd.to_datetime(df["Sample Date"], dayfirst=True, errors="coerce")

# --- 2. Función de colores según niveles de PET -------------

def color_por_pet(value):
    """
    Escala simple para visualizar la intensidad del PET:
    <140      = Bajo (verde)
    140–160   = Medio (amarillo)
    160–180   = Alto (naranja)
    >180      = Muy alto (rojo)
    """
    if value < 140:
        return "green"
    elif value < 160:
        return "yellow"
    elif value < 180:
        return "orange"
    else:
        return "red"

# --- 3. Crear mapa centrado en el dataset --------------------

lat_center = df["Latitude"].mean()
lon_center = df["Longitude"].mean()

m = folium.Map(location=[lat_center, lon_center], zoom_start=5)

# --- 4. Agregar puntos coloreados por PET ---------------------

for _, row in df.iterrows():
    valor = row["pet"]
    color = color_por_pet(valor)

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=(
            f"<b>PET:</b> {valor}<br>"
            f"<b>Fecha:</b> {row['Sample Date'].date()}"
        )
    ).add_to(m)

# --- 5. Agregar leyenda --------------------------------------

legend_html = """
<div style="
    position: fixed;
    bottom: 30px; left: 30px; width: 200px; height: 150px;
    background-color: white; z-index:9999;
    font-size:14px; border:2px solid grey; border-radius:8px; padding:10px;
">
<b>Niveles de PET</b><br>
<i style="background:green; width:10px; height:10px; display:inline-block;"></i> Bajo (&lt;140)<br>
<i style="background:yellow; width:10px; height:10px; display:inline-block;"></i> Medio (140–160)<br>
<i style="background:orange; width:10px; height:10px; display:inline-block;"></i> Alto (160–180)<br>
<i style="background:red; width:10px; height:10px; display:inline-block;"></i> Muy alto (&gt;180)<br>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

# --- 6. Guardar mapa -----------------------------------------

m.save("mapa_pet_terraclimate.html")
print("Mapa guardado como 'mapa_pet_terraclimate.html'")

Mapa guardado como 'mapa_pet_terraclimate.html'
